In [ ]:
#!pip install numpy implicit tqdm

In [ ]:
#!pip install -U scikit-learn

In [ ]:
#!pip install scikit-learn=1.0.2

In [ ]:
#!pip install -U torch

In [ ]:
#!pip install polars==0.19.9 

In [ ]:
#!pip install ray

In [ ]:
#!pip install recbole

In [ ]:
#%pip install MetaUtils

In [1]:
import pickle

In [2]:
with open('list_good_users.pkl', 'rb') as f:
    list_good_users = pickle.load(f)

list_good_users

array(['u_0', 'u_100', 'u_10000', ..., 'u_999994', 'u_999996', 'u_999999'],
      dtype=object)

In [3]:
len(list_good_users)

819386

In [ ]:
import torch
print(torch.__version__)

In [1]:
import polars as pl
import numpy as np
# from tqdm import tqdm
from tqdm.notebook import tqdm   # лучше вывод?
#from scipy.sparse import csr_matrix
#import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100
DATASET_NAME = "hh_recsys"

In [2]:
pl.show_versions()

--------Version info---------
Polars:              0.19.9
Index type:          UInt32
Platform:            Linux-5.10.0-21-amd64-x86_64-with-glibc2.31
Python:              3.9.2 (default, Feb 28 2021, 17:03:44) 
[GCC 10.2.1 20210110]

----Optional dependencies----
adbc_driver_sqlite:  <not installed>
cloudpickle:         <not installed>
connectorx:          <not installed>
deltalake:           <not installed>
fsspec:              2023.6.0
gevent:              <not installed>
matplotlib:          3.7.1
numpy:               1.23.5
openpyxl:            <not installed>
pandas:              1.5.3
pyarrow:             13.0.0
pydantic:            1.10.9
pyiceberg:           <not installed>
pyxlsb:              <not installed>
sqlalchemy:          2.0.20
xlsx2csv:            <not installed>
xlsxwriter:          <not installed>


In [ ]:
print(pl.__version__ == "0.19.9")

In [2]:
import datetime
import os

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.quick_start import run_recbole
from recbole.quick_start.quick_start import load_data_and_model  
from recbole.utils.case_study import full_sort_topk
from recbole.utils.utils import get_trainer

2024-01-27 15:02:47.140471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 15:02:47.140519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 15:02:47.141619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 15:02:47.147784: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-27 15:02:48.008322: W tensorflow/comp

In [5]:
train = pl.read_parquet("./hh_recsys_train_hh.pq", low_memory=True)
test = pl.read_parquet("./hh_recsys_test_hh.pq", low_memory=True)
vacancies = pl.read_parquet("./hh_recsys_vacancies.pq", low_memory=True)

In [4]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_332060""","""s_28301374""","[""v_2571684"", ""v_488179"", … ""v_2633899""]","[2, 2, … 2]","[2023-11-01 00:40:58.105, 2023-11-01 00:58:13.091, … 2023-11-01 01:35:54.456]"
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"


In [6]:
train.select(pl.col("user_id").unique())

user_id
str
"""u_950786"""
"""u_327423"""
"""u_57491"""
"""u_639701"""
"""u_316031"""
"""u_838341"""
"""u_646141"""
"""u_526261"""
"""u_701297"""


In [7]:
#train.filter(pl.col("vacancy_id").list.len()<52).select(pl.col("user_id").unique())  # 879944

train = train.filter(pl.col("vacancy_id").list.len()<52)

In [28]:
# train = train.filter(pl.col("user_id").is_in(list_good_users.tolist()))

In [8]:
train.select(pl.col("vacancy_id").list.len().mean())

vacancy_id
f64
5.142407


In [9]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"
"""u_407319""","""s_23566251""","[""v_268677"", ""v_692096"", … ""v_416576""]","[2, 2, … 2]","[2023-11-01 04:56:41.605, 2023-11-01 04:57:49.326, … 2023-11-01 05:15:18.660]"


In [10]:
test

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]"


In [11]:
test.select(pl.col("vacancy_id").list.len().max())

vacancy_id
u32
26


In [12]:
train = train.vstack(test)

In [13]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"
"""u_407319""","""s_23566251""","[""v_268677"", ""v_692096"", … ""v_416576""]","[2, 2, … 2]","[2023-11-01 04:56:41.605, 2023-11-01 04:57:49.326, … 2023-11-01 05:15:18.660]"


In [14]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,list[str],i64,i64,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""<strong>Обязан…","[""Пользователь ПК"", ""Работа в команде"", … ""PR-консультирование""]",16500,null,"""RUR""","""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""<p><strong>Воз…","[""Активные продажи"", ""Холодные продажи"", … ""Организация мероприятий""]",50000,null,"""RUR""","""a_744""","""ar_2""","""full""","""fullDay""","""noExperience"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""<strong>Обязан…",null,50000,65000,"""RUR""","""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""<p><strong>Для…",null,38500,42000,"""RUR""","""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience"""
"""v_312507""","""Торговый предс…","""c_206699""","""<p>Компания ТД…","[""Продуктивность"", ""Клиентоориентированность"", … ""Развитие продаж""]",60000,null,"""RUR""","""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""<p><strong>X</…","[""Контроль качества"", ""Производственный контроль"", … ""Лабораторная диагностика""]",88000,null,"""RUR""","""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""<p>Если ты ище…","[""Активные продажи"", ""Поиск и привлечение клиентов"", … ""Проведение переговоров""]",30000,100000,"""RUR""","""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""<strong>Обязан…",null,null,null,null,"""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""<em><strong>Ст…",null,100000,null,"""RUR""","""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3"""


In [15]:
vacancies = vacancies.with_columns(pl.col("compensation.from").fill_null(strategy="min"))
vacancies = vacancies.with_columns(pl.when(pl.col("compensation.to").is_null()).then(pl.col("compensation.from")).otherwise(pl.col("compensation.to")).alias("compensation.to"))
vacancies = vacancies.with_columns(pl.col("compensation.currencyCode").fill_null("RUR"))
# не можем это все сделать в одну строку потому что есть строки где и from и to ==  null - тогда они обработаются некорректно

keySkills.keySkill и area.regionId могут быть null, остальные вроде бы нет.
(Проверим:)

In [16]:
vacancies.filter(pl.col("name").is_null())

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,list[str],i64,i64,str,str,str,str,str,str


In [17]:
vacancies = vacancies.with_columns(pl.col("description").str.replace_all("<.*?>", "").  # убираем html-тэги
                                   str.replace_all("&[A-Za-z0-9#]+;", "").   # убираем entity вроде  &quot;
                                   str.to_lowercase())                       # лучше все к нижнему регистру
#                                  str.replace_all("&quot;", "")) # - можно было бы и так, последовательно, но их бывает много разных

In [18]:
vacancies = vacancies.with_columns(pl.col("keySkills.keySkill").list.join(' ').str.to_lowercase())  # ключевые навыки записываем в строку через пробел и в нижнем регистре
# если .join(', ') - то будет через запятую

In [19]:
with pl.Config(fmt_str_lengths=50):
#    print(vacancies['description'])
    print(vacancies['keySkills.keySkill'])

shape: (2_734_129,)
Series: 'keySkills.keySkill' [str]
[
	"пользователь пк работа в команде умение планирова…
	"активные продажи холодные продажи кредитные догов…
	null
	null
	"продуктивность клиентоориентированность управлени…
	"контроль качества производственный контроль навык…
	"активные продажи поиск и привлечение клиентов под…
	null
	null
	"умение работать в команде уборка строительного му…
	"уборка, стирка, домработница, помощница по хозяйс…
	"медицинская документация деловое общение медицинс…
	…
	"работа в команде управление курьерской службой ку…
	"парковочная система бесконфликтность работа в ком…
	null
	"работа в команде быстро учусь желание учиться жел…
	null
	null
	"приемка груза пользователь пк учет товаров"
	"работа с кассой пользователь пк работа в команде …
	"пользователь пк умение работать в команде учет то…
	"желание работать и зарабатывать высокая энергично…
	"контроль исправности оборудования инженерные сист…
	"предпечатная подготовка статистический анализ"
	"1с: то

In [20]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""обязанности: …","""пользователь п…",16500,16500,"""RUR""","""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""возможно, наша…","""активные прода…",50000,50000,"""RUR""","""a_744""","""ar_2""","""full""","""fullDay""","""noExperience"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""обязанности: …",null,50000,65000,"""RUR""","""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""для каждого, к…",null,38500,42000,"""RUR""","""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience"""
"""v_312507""","""Торговый предс…","""c_206699""","""компания тд ол…","""продуктивность…",60000,60000,"""RUR""","""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""x5 ready food …","""контроль качес…",88000,88000,"""RUR""","""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""если ты ищешь …","""активные прода…",30000,100000,"""RUR""","""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""обязанности: …",null,1,1,"""RUR""","""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""строительная к…",null,100000,100000,"""RUR""","""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3"""


In [21]:
vacancies.filter(pl.col("compensation.currencyCode") == 'UZS')

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""v_1222387""","""Специалист слу…","""c_160874""","""пользователи я…",null,5530000,5530000,"""UZS""","""a_3964""",null,"""full""","""remote""","""noExperience"""
"""v_2337785""","""Менеджер по lu…","""c_112093""","""в компании bor…",null,8000000,8000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between3And6"""
"""v_446072""","""Региональный м…","""c_42112""","""обязанности: …",null,8000000,10000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between3And6"""
"""v_540297""","""Ведущий инжене…","""c_173551""","""обязанности: …","""разработка про…",8000000,8000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between3And6"""
"""v_875594""","""Специалист слу…","""c_160874""","""пользователи я…",null,5530000,5530000,"""UZS""","""a_6221""",null,"""full""","""remote""","""noExperience"""
"""v_440966""","""Промоутер-конс…","""c_146958""","""обязанности: …",null,2500000,8700000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""noExperience"""
"""v_1594879""","""Офис-менеджер""","""c_39683""","""в новый соврем…","""розничная торг…",5000000,5000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between1And3"""
"""v_2220635""","""Менеджер по пр…","""c_243045""","""яндекс.еда – э…","""деловое общени…",1,7200000,"""UZS""","""a_5141""",null,"""full""","""remote""","""between1And3"""
"""v_849781""","""Менеджер по ра…","""c_104754""","""компания vian …","""клиентоориенти…",6500000,6500000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between1And3"""


Нужно свести все валюты к одной или исключить вакансии с зарплатой не в рублях!

In [22]:
vacancies.select(['compensation.currencyCode']).unique()

compensation.currencyCode
str
"""EUR"""
"""RUR"""
"""UAH"""
"""KZT"""
"""UZS"""
"""BYR"""
"""KGS"""
"""GEL"""
"""AZN"""


In [23]:
currency_rates = {"UZS": 0.007269,
                  "KGS": 1.01,
                  "USD": 89.69,
                  "GEL": 33.3,
                  "BYR": 28.23,
                  "AZN": 52.76,
                  "KZT": 0.197708,
                  "UAH": 2.36,
                  "RUR": 1.0,
                  "EUR": 99.19
                  }

Преобразуем суммы зарплат в соответствии с курсом ЦБ (на 09.01.2023)

In [24]:
vacancies = vacancies.rename({"compensation.currencyCode": "currencyRate", "compensation.from": "compensation_from", "compensation.to": "compensation_to"})

#vacancies = vacancies.with_columns(currencyRate=pl.col("currencyRate").replace(currency_rates, default=0.0))   # для версии 0.20
vacancies = vacancies.with_columns(currencyRate=pl.col("currencyRate").map_dict(currency_rates, default=0.0))   # для версии 0.19


vacancies = vacancies.with_columns(compensation_from = pl.col('compensation_from')*pl.col('currencyRate'))
vacancies = vacancies.with_columns(compensation_to = pl.col('compensation_to')*pl.col('currencyRate'))

In [25]:
vacancies.filter(pl.col("vacancy_id") == 'v_446072')  # посмотрим что получилось для вакансии с зарплатой в UZS

vacancy_id,name,company.id,description,keySkills.keySkill,compensation_from,compensation_to,currencyRate,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,str,f64,f64,f64,str,str,str,str,str
"""v_446072""","""Региональный м…","""c_42112""","""обязанности: …",null,58152.0,72690.0,0.007269,"""a_3964""",null,"""full""","""fullDay""","""between3And6"""


In [26]:
vacancies = vacancies.with_columns(salary_from=(pl.col("compensation_from").log10()*10).floor().cast(pl.Int64).cast(pl.Utf8)                 # Переведем зарплату в символьный вид
                                   ).with_columns(salary_to=(pl.col("compensation_to").log10()*10).floor().cast(pl.Int64).cast(pl.Utf8))     # потому что SASRecF не умеет с float

In [27]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation_from,compensation_to,currencyRate,area.id,area.regionId,employment,workSchedule,workExperience,salary_from,salary_to
str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""обязанности: …","""пользователь п…",16500.0,16500.0,1.0,"""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience""","""42""","""42"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""возможно, наша…","""активные прода…",50000.0,50000.0,1.0,"""a_744""","""ar_2""","""full""","""fullDay""","""noExperience""","""46""","""46"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""обязанности: …",null,50000.0,65000.0,1.0,"""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6""","""46""","""48"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""для каждого, к…",null,38500.0,42000.0,1.0,"""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience""","""45""","""46"""
"""v_312507""","""Торговый предс…","""c_206699""","""компания тд ол…","""продуктивность…",60000.0,60000.0,1.0,"""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3""","""47""","""47"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""x5 ready food …","""контроль качес…",88000.0,88000.0,1.0,"""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3""","""49""","""49"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""если ты ищешь …","""активные прода…",30000.0,100000.0,1.0,"""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3""","""44""","""50"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""обязанности: …",null,1.0,1.0,1.0,"""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6""","""0""","""0"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""строительная к…",null,100000.0,100000.0,1.0,"""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3""","""50""","""50"""


In [28]:
vacancies.select(['employment']).unique()

employment
str
"""full"""
"""project"""
"""part"""
"""probation"""
"""volunteer"""


# SASRec

https://recbole.io/docs/user_guide/data_intro.html

https://recbole.io/docs/user_guide/data/atomic_files.html

https://recbole.io/docs/user_guide/usage/use_modules.html

https://www.recbole.io/docs/user_guide/model/sequential/sasrec.html

In [29]:
'''
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
'''

action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

pairs = train.select(['user_id', 'vacancy_id', 'action_type', 'action_dt']).explode(['vacancy_id', 'action_type', 'action_dt'])

In [30]:
pairs

user_id,vacancy_id,action_type,action_dt
str,str,i64,datetime[ns]
"""u_1057881""","""v_665861""",2,2023-11-01 00:23:51.452
"""u_1036784""","""v_2594840""",2,2023-11-01 00:52:34.023
"""u_786220""","""v_1473781""",1,2023-11-01 00:58:20.793
"""u_786220""","""v_1622905""",2,2023-11-01 01:17:04.774
"""u_786220""","""v_1621959""",2,2023-11-01 00:54:00.360
"""u_786220""","""v_2289180""",2,2023-11-01 01:16:05.415
"""u_786220""","""v_1473781""",2,2023-11-01 00:55:32.149
"""u_786220""","""v_1719419""",2,2023-11-01 01:09:54.498
"""u_786220""","""v_2654062""",2,2023-11-01 01:00:50.537


In [31]:
def conv_date(source_date):
    return datetime.datetime.timestamp(source_date)

In [32]:
if not os.path.exists(DATASET_NAME):
    os.mkdir(DATASET_NAME)

In [33]:
# для версии 0.19

p = pairs.with_columns(action_dt=pl.col("action_dt").map_elements(conv_date)
                   ).with_columns(action_type=pl.col("action_type").map_dict(action_weights, default=0.0)
                                  ).rename({"user_id": "user_id:token", "vacancy_id": "vacancy_id:token", "action_type": "action_type:float", "action_dt": "timestamp:float"}
                                  ).write_csv("./" + DATASET_NAME + "/hh_recsys.inter", separator="\t")


In [ ]:
# для версии 0.20

pairs.with_columns(action_dt=pl.col("action_dt").map_elements(conv_date)
                   ).with_columns(action_type=pl.col("action_type").replace(action_weights, default=0.0)
                                  ).rename({"user_id": "user_id:token", "vacancy_id": "vacancy_id:token", "action_type": "action_type:float", "action_dt": "timestamp:float"}
                                           ).write_csv("./" + DATASET_NAME + "/hh_recsys.inter", include_header=True, separator="\t")

In [34]:
# для версии 0.19

vacancies.rename({"vacancy_id": "vacancy_id:token", "name": "name:token_seq", "company.id": "company_id:token", "description": "description:token_seq", 
                                 "keySkills.keySkill": "keySkills:token_seq", "compensation_from": "compensation_from:float", "compensation_to": "compensation_to:float", 
                                 "currencyRate": "currencyRate:float", "area.id": "area_id:token", "area.regionId": "area_regionId:token", 
                                 "employment": "employment:token", "workSchedule": "workSchedule:token", "workExperience": "workExperience:token",
                                 "salary_from": "salary_from:token", "salary_to": "salary_to:token"}                                 
                                 ).write_csv("./" + DATASET_NAME + "/hh_recsys.item", separator="\t")

In [ ]:
# для версии 0.20

vacancies.with_columns(pl.col("keySkills.keySkill").cast(pl.List(pl.Utf8)).list.join(", ")
                       ).rename({"vacancy_id": "vacancy_id:token", "name": "name:token_seq", "company.id": "company_id:token", "description": "description:token_seq", 
                                 "keySkills.keySkill": "keySkills:token_seq", "compensation_from": "compensation_from:float", "compensation_to": "compensation_to:float", 
                                 "currencyRate": "currencyRate:float", "area.id": "area_id:token", "area.regionId": "area_regionId:token", 
                                 "employment": "employment:token", "workSchedule": "workSchedule:token", "workExperience": "workExperience:token",
                                 "salary_from": "salary_from:token", "salary_to": "salary_to:token"}
                                 ).write_csv("./" + DATASET_NAME + "/hh_recsys.item", include_header=True, separator="\t")

Можно начинать с этой точки, если датасет уже сформирован

In [3]:
parameter_dict = {
    'data_path': './',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'vacancy_id',
    'RATING_FIELD': 'action_type',
    'TIME_FIELD': 'timestamp',
#    'MAX_ITEM_LIST_LENGTH': 30,  # для Caser    
    'user_inter_num_interval': "[10,inf)",
    'item_inter_num_interval': "[15,inf)",
    'seq_len': {'vacancy_id': 10}, # попробуем последовательности не больше 10. Так ли???
    'MAX_ITEM_LIST_LENGTH': 10,    
    'load_col': {'inter': ['user_id', 'vacancy_id', 'action_type', 'timestamp'],
#                 'user': ['user_id'],
                 'item': ['vacancy_id', 'name', 'company_id', 'keySkills', 'salary_from', 'salary_to', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ]},  # для SASRecF берем salary а не compensation
#                 'item': ['vacancy_id', 'name', 'company_id', 'keySkills', 'compensation_from', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ]},  # не все, но на большее не хватает памяти
#    'selected_features': ['name', 'company_id', 'keySkills', 'compensation_from', 'compensation_to', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ],
#    'selected_features': ['name', 'company_id', 'keySkills'], - ок
#    'selected_features': ['compensation_from', 'compensation_to'], # mat1 and mat2 shapes cannot be multiplied (10x256 and 320x64)  или RuntimeError: torch.cat(): expected a non-empty list of Tensors
#    'selected_features': ['name', 'company_id', 'keySkills', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ],
    'selected_features': ['name', 'company_id', 'keySkills', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience', 'salary_from', 'salary_to'],
#    'numerical_features': ['compensation_from', 'compensation_to'],
#    'normalize_all': True,
    'neg_sampling': None,
    'train_neg_sample_args': None,
    'worker': 8,   # на vm66
    'train_batch_size': 1024,
    'eval_batch_size': 1024,
    'epochs': 3,
    'use_gpu': False,
#    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'metrics': ['Recall', 'MRR'],
    'loss_type': 'CE',
#    'train_neg_sample_args': {'uniform': 1},  # для TransRec  
#    'loss_type': 'BPR',  # для FPMC
#    'train_neg_sample_args': {'distribution': 'uniform', 'sample_num': 1},  # для FPMC
#    'reproducibility': False,   # для HRM
    'topk': 100,
    'valid_metric': 'MRR@100',
    "stopping_step": 2, # останавливаем обучение если в течение этого кол-ва эпох не было улучшения валидационной метрики
    'hidden_size': 64,  # по дефолту 64
    'inner_size': 256,  # по дефолту 256
    'hidden_dropout_prob': 0.3,  # 0.5
    'attn_dropout_prob': 0.3,    # 0.5
    'seed': 42,
    'eval_args': {
        'split': {'RS': [0.95, 0.03, 0.02]},  # 8,1,1 ?
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}  #  labeled ??
}

train_batch_size: 2048  по дефолту<br>
eval_batch_size: 4096

In [ ]:
config = Config(model='SASRec', dataset=DATASET_NAME, config_dict=parameter_dict)

In [ ]:
config = Config(model='SLIMElastic', dataset=DATASET_NAME, config_dict=parameter_dict)

In [ ]:
hh_recsys_dataset = create_dataset(config)

In [ ]:
hh_recsys_dataset # это 30 и 50, датасет объединенный с test

In [ ]:
# hh_recsys_dataset # это 10 и 15 и цепочки max 10, датасет объединенный с test

In [ ]:
# hh_recsys_dataset  # это 10 и 15

In [ ]:
# hh_recsys_dataset   # это полный

In [ ]:
# train_data, valid_data, test_data = data_preparation(config, hh_recsys_dataset)   # работает 8 минут

In [4]:
run_recbole(model='SASRecF', dataset=DATASET_NAME, config_dict=parameter_dict)

Train     1:  38%|███████████████▊                          | 3729/9926 [1:58:37<2:14:16,  1.30s/it]:  38%|███████████████▊                          | 3730/9926 [1:58:39<2:13:17,  1.29s/it]:  38%|███████████████▊                          | 3731/9926 [1:58:40<2:12:42,  1.29s/it]:  38%|███████████████▊                          | 3732/9926 [1:58:41<2:13:27,  1.29s/it]:  38%|███████████████▊                          | 3733/9926 [1:58:43<2:12:47,  1.29s/it]:  38%|███████████████▊                          | 3734/9926 [1:58:44<2:12:02,  1.28s/it]:  38%|███████████████▊                          | 3735/9926 [1:58:45<2:12:19,  1.28s/it]:  38%|███████████████▊                          | 3736/9926 [1:58:46<2:12:00,  1.28s/it]:  38%|███████████████▊                          | 3737/9926 [1:58:48<2:11:15,  1.27s/it]:  38%|███████████████▊                          | 3738/9926 [1:58:49<2:11:53,  1.28s/it]:  38%|███████████████▊                          | 3739/9926 [1:58:50<2:11:06,  1.27s/it]:  38%|███

{'best_valid_score': 0.1598,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@100', 0.5666),
              ('mrr@100', 0.1598)]),
 'test_result': OrderedDict([('recall@100', 0.5823), ('mrr@100', 0.1802)])}

Надо будет использовать full_sort_topk ?

как описано тут: https://recbole.io/docs/user_guide/usage/case_study.html  <br>
и тут: https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

Или методы модели https://recbole.io/docs/recbole/recbole.model.sequential_recommender.sasrec.html

In [ ]:
samples = pl.read_parquet("./hh_recsys_sample.pq")

In [ ]:
samples

в т.ч. можно использовать для загрузки при обрыве

In [ ]:
!ls saved

Следующая ячейка выполняется 12 минут.

In [ ]:
import torch

In [ ]:
model_cpu = torch.load('models-gpu/HRM-Jan-11-2024_18-18-49.pth', map_location=torch.device('cpu'))

In [ ]:
torch.save(model_cpu, "saved/HRM-Jan-11-2024_18-18-49.pth")

In [7]:

config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='saved/SASRecF-Jan-27-2024_15-13-17.pth',
)

28 Jan 10:07    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 42
state = INFO
reproducibility = True
data_path = ./hh_recsys
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 3
train_batch_size = 1024
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.95, 0.03, 0.02]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR']
topk = [100]
valid_metric = MRR@100
valid_metric_bigger = True
eval_batch_size = 1024
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_sepa

In [ ]:
model_file='saved/HRM-Jan-11-2024_18-18-49.pth'

In [ ]:
import torch

In [ ]:
from recbole.utils import init_seed, init_logger, get_model
from logging import getLogger

In [ ]:
# то же самое что load_data_and_model, но вручную и с подменой device
checkpoint = torch.load(model_file)
config = checkpoint["config"]

config["use_gpu"] = False  #  вот и проверим
config["device"] = torch.device("cpu")

init_seed(config["seed"], config["reproducibility"])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

init_seed(config["seed"], config["reproducibility"])
#model = get_model(config["model"])(config, train_data._dataset).to(config["device"])
model = get_model(config["model"])(config, train_data._dataset).to(torch.device("cpu"))
model.load_state_dict(checkpoint["state_dict"])
model.load_other_parameter(checkpoint.get("other_parameter"))

In [10]:
config


General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 42
state = INFO
reproducibility = True
data_path = ./hh_recsys
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 3
train_batch_size = 1024
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.95, 0.03, 0.02]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR']
topk = [100]
valid_metric = MRR@100
valid_metric_bigger = True
eval_batch_size = 1024
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIEL

In [11]:
config['epochs']=5

In [12]:

# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# resume from break point
checkpoint_file = 'saved/SASRecF-Jan-27-2024_15-13-17.pth'
trainer.resume_checkpoint(checkpoint_file)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, show_progress=True)


Train     3:   2%|▋                                            | 152/9926 [05:09<8:58:45,  3.31s/it]:   2%|▋                                            | 153/9926 [05:13<9:55:17,  3.65s/it]:   2%|▋                                            | 154/9926 [05:16<9:30:34,  3.50s/it]:   2%|▋                                            | 155/9926 [05:19<9:00:53,  3.32s/it]:   2%|▋                                            | 156/9926 [05:23<9:11:37,  3.39s/it]:   2%|▋                                            | 157/9926 [05:24<7:28:21,  2.75s/it]:   2%|▋                                            | 158/9926 [05:25<6:21:56,  2.35s/it]:   2%|▋                                            | 159/9926 [05:28<6:33:07,  2.42s/it]:   2%|▋                                            | 160/9926 [05:29<5:40:22,  2.09s/it]:   2%|▋                                            | 161/9926 [05:31<5:07:54,  1.89s/it]:   2%|▋                                            | 162/9926 [05:33<5:39:50,  2.09s/it]:   2%|▋  

In [ ]:
dataset.token2id(dataset.uid_field, 'u_332060')

In [ ]:
dataset.token2id(dataset.uid_field, 'u_485235')

In [ ]:
train.filter(pl.col("user_id") == 'u_700997')

In [ ]:
train.filter(pl.col("user_id") == 'u_485235')

In [ ]:
test.filter(pl.col("user_id") == 'u_485235')

In [ ]:
test.filter(pl.col("user_id") == 'u_700997')

In [ ]:
'u_482520' in dataset.field2token_id[dataset.uid_field]

In [5]:
test = pl.read_parquet("./hh_recsys_test_hh.pq", low_memory=True)
#vacancies = pl.read_parquet("./hh_recsys_vacancies.pq")

In [6]:
test_users = test['user_id'].to_list()
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [ ]:
dataset.field2token_id[dataset.uid_field]

In [ ]:
test_users[1]

In [ ]:
'u_684868' in dataset.field2token_id[dataset.uid_field]

In [ ]:
test_users[12]

In [ ]:
test_vacancies[13]

In [ ]:
cuser = dataset.token2id(dataset.uid_field, 'u_186164') 
cuser

In [ ]:
scores, topk_iid_list = full_sort_topk([cuser], model, test_data, k=10, device=config['device'])

In [ ]:
topk_iid_list[-1]

In [ ]:
last_topk_iid_list = topk_iid_list[-1]

In [ ]:
recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

In [ ]:
recommendations

In [ ]:
bad_list = ['u_186164']

Это для моделей из implicit

https://benfred.github.io/implicit/api/models/recommender_base.html#implicit.recommender_base.RecommenderBase

М.б. model.recommend(userid, user_items, N=10, filter_already_liked_items=True, filter_items=None, recalculate_user=False, items=None)

recalculate_user=True, filter_already_liked_items=False (?)

In [8]:
predictions = []
num_not_found = 0
zero_topk = 0

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
#    print("Try: ", user)
    if user not in dataset.field2token_id[dataset.uid_field]:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?)
        predictions.append(vacs)
        num_not_found += 1
#        print("Not found user: ", user)
        
        continue
    
    cuser = dataset.token2id(dataset.uid_field, user)   # получили id очередного юзера по его имени
    

    _, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device=config['device'])
#    _, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device='cpu')
    if len(topk_iid_list) == 0:
#        print("Zero topk for user ", user)
        predictions.append(vacs)
        zero_topk += 1
        continue
    
    last_topk_iid_list = topk_iid_list[-1]
    recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

    predictions.append(recommendations)

print("Not found: ", num_not_found)
print("Zero topk: ", zero_topk)


  0%|          | 0/83189 [00:00<?, ?it/s]

Not found:  30906
Zero topk:  0


Для юзеров, которые есть в тесте, но не попали в трейн, можно:
- тупо брать те вакансии что они смотрели
- использовать ALS
- использовать какую-нибудь другую модель
- как-то находить самую похожую последовательность вакансий и предсказывать для такого юзера?  или искать такую в тесте?.. ???

Как придумать предсказание на основе последовательности для любого юзера, независимо от его id?..


In [9]:
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('sasrecf-10-15_good-dataset_submission.pq')

In [ ]:
score, topk_iid_list = full_sort_topk([1], model, test_data, k=6, device=config['device'])
last_topk_iid_list = topk_iid_list[-1]
external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
external_item_list

In [ ]:
topk_iid_list.shape

In [ ]:
score

In [ ]:
dataset.uid_field

In [ ]:
type(dataset)

In [ ]:
dir(dataset)

In [ ]:
dataset.history_item_matrix(value_field='1')

In [ ]:
dataset[dataset.uid_field]

In [ ]:
index = np.isin(dataset[dataset.uid_field].numpy(), [1])
input_interaction = dataset[index]
input_interaction

In [ ]:
input_features = test_data.dataset[np.isin(test_data.dataset[test_data.dataset.uid_field].numpy(), [2])]
input_features

## Попробуем другую модель

In [ ]:
config_fossil = Config(model='FOSSIL', dataset=DATASET_NAME, config_dict=parameter_dict)

In [ ]:
hh_recsys_dataset_bpr = create_dataset(config_fossil)

In [ ]:
run_recbole(model='SHAN', dataset=DATASET_NAME, config_dict=parameter_dict)

In [ ]:
external_user_ids = dataset.id2token(dataset.uid_field, list(range(dataset.user_num)))[1:]   #fist element in array is 'PAD'(default of Recbole) ->remove it 

In [ ]:
external_user_ids

In [ ]:
external_item_list

In [ ]:
train = pl.read_parquet("./hh_recsys_train_hh.pq")
test = pl.read_parquet("./hh_recsys_test_hh.pq")

In [ ]:
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
idx2user = {idx: user_id for user_id, idx in user2idx.items()}


In [ ]:
test_users = test['user_id'].to_list()
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [ ]:
predictions = []

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    if user not in user2idx:
        predictions.append(vacs)
        continue
    cuser = user2idx[user]

    recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS)[0]
    recommendations = [idx2vac[cv] for cv in recommendations]
    predictions.append(recommendations)

In [ ]:
#test = test.with_columns(pl.lit(predictions).alias('predictions'))
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')

In [ ]:
test

Поэкспериментируем с SASRec

In [ ]:
run_recbole(model='BPR', dataset='ml-100k')

In [ ]:
df = pl.DataFrame({
    'id': [1, 2, 3],
    'name': ['Mercy', 'Tomi', 'Grace'],
    'hobbies': [['reading', 'cooking'], ['running'], ['traveling', 'baking']],
    'favorites': [['pizza', 'ice cream'], ['bread'], ['cake', 'hamburger']],
    'test': [['1', '2'], ['3'], ['4', '5']]
})

In [ ]:
df

In [ ]:
df_exploded = df.explode(('hobbies', 'favorites', 'test'))
df_exploded

In [ ]:
s = "2023-11-01 00:58:13.091"

In [ ]:
datetime.datetime.timestamp(datetime.datetime.fromisoformat(s))

In [ ]:
1698825493.091